<a href="https://colab.research.google.com/github/advxit/EDGE-AI-/blob/main/Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snntorch==0.5.0
!pip install torch
!pip install numpy

!pip install matplotlib

### Cell 1: Installation of Libraries

This cell is responsible for installing the necessary Python libraries required for the project. Each `!pip install` command executes a shell command to install a specific package:

*   `!pip install snntorch==0.5.0`: Installs `snntorch`, a Python library for building and simulating spiking neural networks (SNNs) with PyTorch. The `==0.5.0` specifies a particular version to ensure compatibility.
*   `!pip install torch`: Installs PyTorch, an open-source machine learning framework used for deep learning applications, including building and training neural networks.
*   `!pip install numpy`: Installs NumPy, the fundamental package for numerical computing in Python, providing support for large, multi-dimensional arrays and matrices, along with a collection of high-level mathematical functions to operate on these arrays.
*   `!pip install matplotlib`: Installs Matplotlib, a comprehensive library for creating static, animated, and interactive visualizations in Python.

### Cell 2: Importing Essential Modules

This cell imports various modules and functions from the previously installed libraries. These imports make specific functionalities available for use in the subsequent code:

*   `import torch.nn as nn`: Imports the `nn` module from PyTorch, which contains classes for building neural network layers (like `Linear`, `Conv2d`, etc.), activation functions, and loss functions.
*   `import torch`: Imports the core PyTorch library, providing fundamental tensor operations and utilities.
*   `from torch.utils.data import DataLoader`: Imports `DataLoader` from PyTorch's `utils.data` module. `DataLoader` is used for iterating over datasets in batches, shuffling data, and parallelizing data loading.
*   `from torchvision import datasets, transforms`: Imports `datasets` and `transforms` from `torchvision`. `datasets` provides access to common datasets (like MNIST), and `transforms` offers common image transformations (like converting to tensor, normalization).
*   `import numpy as np`: Imports NumPy and aliases it as `np`, a common convention.
*   `import matplotlib.pyplot as plt`: Imports the `pyplot` module from Matplotlib and aliases it as `plt`, used for plotting and visualization.
*   `import snntorch as snn`: Imports the `snntorch` library and aliases it as `snn`, making its spiking neural network components easily accessible.

### Cell 3: Leaky Integrate-and-Fire (LIF) Neuron Initialization and Input Signal

This cell sets up a single Leaky Integrate-and-Fire (LIF) neuron model and defines an input signal for it:

*   `lif = snn.Leaky(beta=0.95)`: Initializes a `Leaky` neuron from the `snntorch` library. The `beta` parameter (0.95) represents the decay rate of the membrane potential. A `beta` of 0.95 means that the membrane potential will decay by 5% at each time step if there's no input.
*   `num_steps = 200`: Defines the number of simulation steps for which the LIF neuron will be observed. This means the neuron will be simulated for 200 discrete time steps.
*   `x = torch.cat((torch.zeros(30), torch.ones(40)*0.1, torch.zeros(130)))`: Creates a 1D PyTorch tensor `x` which will serve as the input current to the LIF neuron over time. It's composed of:
    *   `torch.zeros(30)`: 30 time steps of zero input.
    *   `torch.ones(40)*0.1`: 40 time steps of a constant input current of 0.1.
    *   `torch.zeros(130)`: 130 time steps of zero input again.
    This creates a pulsed input signal: no input for 30 steps, a small positive current for 40 steps, and then no input for the remaining 130 steps.

### Cell 4: Initializing Membrane Potential and Spike Output

This cell initializes the internal state variables of the LIF neuron:

*   `mem = torch.zeros(1)`: Initializes the membrane potential (`mem`) of the neuron to zero. The `torch.zeros(1)` creates a tensor of size 1 containing a single zero. This represents the initial voltage across the neuron's membrane.
*   `spk = torch.zeros(1)`: Initializes the spike output (`spk`) of the neuron to zero. This tensor will hold whether the neuron spiked (typically 1) or not (0) at a given time step.

### Cell 5: Simulating the Leaky Integrate-and-Fire (LIF) Neuron

This cell contains a loop that simulates the behavior of the LIF neuron over the `num_steps` defined earlier:

*   `mem_rec = []`: Initializes an empty list to store the membrane potential at each time step.
*   `spk_rec = []`: Initializes an empty list to store the spike output at each time step.
*   `for step in range(num_steps):`: This loop iterates `num_steps` (200 times in this case), simulating each discrete time step.
    *   `spk, mem = lif(x[step], mem)`: This is the core line. It calls the `lif` neuron object with the current input `x[step]` and the previous membrane potential `mem`. The `lif` neuron's `forward` method (implicitly called here) calculates the new spike output (`spk`) and the updated membrane potential (`mem`) based on the input current, the previous membrane potential, and its internal parameters (like `beta`).
    *   `mem_rec.append(mem.item())`: The updated membrane potential `mem` (which is a tensor) is extracted as a Python number (`.item()`) and appended to the `mem_rec` list.
    *   `spk_rec.append(spk.item())`: Similarly, the spike output `spk` is extracted as a Python number and appended to the `spk_rec` list.

After this loop, `mem_rec` will contain the membrane potential trajectory, and `spk_rec` will contain the spike train (sequence of 0s and 1s indicating firing) of the LIF neuron in response to the input `x`.

### Cell 6: Converting Recorded Data to PyTorch Tensors

This cell converts the lists of recorded membrane potentials and spikes into PyTorch tensors:

*   `mem_rec = torch.tensor(mem_rec)`: Converts the `mem_rec` list (which holds Python floats) into a PyTorch tensor. This is done to enable further tensor operations and compatibility with PyTorch's computational graph.
*   `spk_rec = torch.tensor(spk_rec)`: Similarly, converts the `spk_rec` list into a PyTorch tensor. Using tensors is crucial for leveraging PyTorch's optimized operations, especially on GPUs.

### Cell 7: Data Loading and Device Configuration

This cell sets up crucial parameters for data loading and specifies the computational device:

*   `batch_size = 128`: Defines the number of samples that will be processed in one batch during training. Using batches helps in efficient training and memory management.
*   `data_path ='/data/mnist'`: Specifies the directory where the MNIST dataset will be downloaded and stored. It's a common practice to use a dedicated directory for datasets.
*   `dtype =torch.float`: Sets the default data type for tensors to `torch.float` (32-bit floating-point numbers). This is a standard precision for many deep learning tasks.
*   `device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")`: This line dynamically selects the computing device. If a CUDA-enabled GPU is available, it will use `'cuda'` for faster computations; otherwise, it defaults to `'cpu'`. This ensures that the code can run on systems with or without a GPU.

### Cell 8: Data Transformation and MNIST Dataset Loading

This cell prepares the MNIST dataset by defining transformations and loading the training and testing splits:

*   `transform = transforms.Compose([...])`: Creates a sequence of transformations to be applied to the MNIST images. `transforms.Compose` chains multiple transformations together:
    *   `transforms.ToTensor()`: Converts a PIL Image or NumPy array to a PyTorch `FloatTensor` and scales the pixel values from the range \[0, 255\] to \[0.0, 1.0\].
    *   `transforms.Normalize((0.1307,), (0.3081,))`: Normalizes the tensor image with a mean of 0.1307 and a standard deviation of 0.3081. These values are common for the MNIST dataset and help in stabilizing training.
    *   `transforms.Grayscale()`: Converts the image to grayscale. MNIST images are already grayscale, but this ensures consistency in case of any variations.
    *   `transforms.Normalize((0,), (1,))`: This normalization step with mean 0 and std dev 1 essentially scales the data to have a unit variance and zero mean *after* the previous normalization. For grayscale images that are already 0-1, this might be redundant or could be intended to ensure pixel values are correctly distributed for a specific SNN input type.
*   `mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)`: Loads the MNIST training dataset. It specifies the `data_path`, `train=True` for the training split, `download=True` to download if not already present, and applies the defined `transform`.
*   `mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)`: Loads the MNIST test dataset, similar to the training set but with `train=False`.

### Cell 9: Creating DataLoaders for Training and Testing

This cell creates `DataLoader` objects, which are essential for iterating through the datasets in mini-batches during training and evaluation:

*   `train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)`:
    *   `mnist_train`: The dataset to load data from.
    *   `batch_size=batch_size`: Specifies the number of samples per batch (128, as defined in Cell 7).
    *   `shuffle=True`: The data will be shuffled at the beginning of each epoch. This is crucial for training to prevent the model from learning the order of samples.
    *   `drop_last=True`: If the dataset size is not perfectly divisible by the batch size, the last incomplete batch will be dropped. This ensures all batches have the same size, which can simplify model architecture and training logic.
*   `test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last = True)`:
    *   Similar to `train_loader`, but uses the `mnist_test` dataset. Shuffling and dropping the last batch are also applied here, which is common for consistent evaluation.

### Cell 10: Defining Spiking Neural Network (SNN) Parameters

This cell defines various hyper-parameters and network architecture parameters that will be used for building the Spiking Neural Network:

*   `num_inputs = 784`: This represents the number of input features. For MNIST images (28x28 pixels), 28 * 28 = 784. Each pixel will be an input to the first layer of the network.
*   `num_hidden = 1000`: Defines the number of neurons in the hidden layer of the network. This is a common practice to have a larger hidden layer for feature extraction.
*   `num_outputs = 10`: Represents the number of output classes. For MNIST, there are 10 digits (0-9), so there are 10 output neurons, each corresponding to a digit.
*   `num_steps = 25`: This parameter specifies the number of simulation steps (time steps) over which the SNN will process each input sample. In SNNs, computation unfolds over time.
*   `beta = 0.95`: This is the decay rate for the membrane potential of the LIF neurons used in the network layers, as seen in Cell 3. A higher `beta` means slower decay.

### Cell 11: Spiking Neural Network (SNN) Model Definition

This cell defines the `Net` class, which is a PyTorch `nn.Module` representing the spiking neural network architecture:

*   `class Net(nn.Module):` Defines a neural network class inheriting from `nn.Module`.
*   `def __init__(self):`: The constructor method where network layers and SNN specific components are initialized.
    *   `super().__init__()`: Calls the constructor of the parent `nn.Module` class.
    *   `self.fc1 = nn.Linear(num_inputs, num_hidden)`: Defines the first fully connected (linear) layer. It takes `num_inputs` (784) and outputs `num_hidden` (1000) features.
    *   `self.lif1 = snn.Leaky(beta=beta)`: Instantiates a Leaky Integrate-and-Fire neuron layer from `snntorch` for the first layer, using the global `beta`.
    *   `self.fc2 = nn.Linear(num_hidden, num_outputs)`: Defines the second fully connected layer, taking `num_hidden` features and outputting `num_outputs` (10) features.
    *   `self.lif2 = snn.Leaky(beta=beta)`: Instantiates another Leaky Integrate-and-Fire neuron layer for the second layer.
    *   `self.numsteps = num_steps`: Stores the `num_steps` parameter as an attribute of the network.
*   `def forward(self, x):`: The `forward` method defines how data flows through the network.
    *   `mem1 = self.lif1.init_leaky()`: Initializes the membrane potential for the first LIF layer. `init_leaky()` is a `snntorch` method to get a zero-initialized membrane potential.
    *   `mem2 = self.lif2.init_leaky()`: Initializes the membrane potential for the second LIF layer.
    *   `spk2_rec = []` and `mem2_rec = []`: Lists to record the spike outputs and membrane potentials of the second (output) LIF layer over time.
    *   `for step in range(num_steps):`: This loop simulates the SNN's behavior over `num_steps` for each input. This is where the temporal dynamics of SNNs are handled.
        *   `cur11 = self.fc1(x.flatten(1))`: The input `x` (likely a batch of images) is first flattened from its spatial dimensions (e.g., `batch_size, 1, 28, 28` to `batch_size, 784`) and then passed through the first linear layer (`self.fc1`) to produce input current `cur11` for the first LIF layer.
        *   `spk1, mem1 = self.lif1(cur1, mem1)`: **Correction needed**: There is a typo here. It should be `cur11` instead of `cur1`. This line passes the current from `fc1` (`cur11`) and the previous membrane potential `mem1` through the first LIF neuron layer. It returns the new spikes `spk1` and updated membrane potential `mem1`.
        *   `cur2 = self.fc2(spk1)`: The spikes (`spk1`) from the first LIF layer act as input current for the second linear layer (`self.fc2`).
        *   `spk2, mem2 = self.lif2(cur2, mem2)`: The current from `fc2` (`cur2`) and the previous membrane potential `mem2` are passed through the second LIF neuron layer, producing output spikes `spk2` and updated `mem2`.
        *   `spk2_rec.append(spk2)` and `mem2_rec.append(mem2)`: The output spikes and membrane potentials from the second layer are recorded at each time step.
    *   `return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)`: After the simulation loop, the recorded lists of spikes and membrane potentials are stacked into single tensors along a new dimension (dimension 0), making them `(num_steps, batch_size, num_outputs)` shaped tensors. These represent the time-series output of the SNN.
*   `net = Net().to(device)`: An instance of the `Net` class is created and then moved to the specified `device` (CPU or GPU) for computation.

### Cell 12: Empty Cell

This cell is currently empty. It might have been intended for future code or is a remnant from previous editing. It has no functional impact on the notebook's execution.

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import snntorch as snn




In [ ]:
lif = snn.Leaky(beta=0.95)
num_steps = 200
x = torch.cat((torch.zeros(30), torch.ones(40)*0.1, torch.zeros(130)))


In [ ]:
mem = torch.zeros(1)
spk = torch.zeros(1)



In [ ]:
mem_rec = []
spk_rec = []

for step in range(num_steps):
    spk, mem = lif(x[step], mem)
    mem_rec.append(mem.item())
    spk_rec.append(spk.item())


In [ ]:
mem_rec = torch.tensor(mem_rec)
spk_rec = torch.tensor(spk_rec)


In [ ]:
from torch import device
batch_size = 128
data_path ='/data/mnist'
dtype =torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
            transforms.Grayscale(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.46MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.23MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.08MB/s]


In [ ]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True, drop_last = True)

In [ ]:
num_inputs = 784
num_hidden = 1000
num_outputs = 10

num_steps = 25
beta = 0.95 #membrane potential decreases by 5%

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)
        self.lif2 = snn.Leaky(beta=beta)
        self.numsteps = num_steps

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        spk2_rec = []
        mem2_rec = []

        for step in range(num_steps):
          cur11 = self.fc1(x.flatten(1))
          spk1, mem1 = self.lif1(cur1, mem1)
          cur2 = self.fc2(spk1)
          spk2, mem2 = self.lif2(cur2, mem2)
          spk2_rec.append(spk2)
          mem2_rec.append(mem2)

        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)
net = Net().to(device)